In [1]:
# First step, import libraries.
import numpy as np 
import pandas as pd 
from matplotlib import pyplot as plt


In [ ]:
# Import the dataset and encode the date
df = pd.read_csv('C:/Users/Divya Admin/Downloads/bitcoin-historical-data/bitstampUSD_1-min_data_2012-01-01_to_2018-01-08.csv')
df['date'] = pd.to_datetime(df['Timestamp'],unit='s').dt.date
group = df.groupby('date')
Real_Price = group['Weighted_Price'].mean()


In [7]:
# split data
prediction_days = 30
df_train= Real_Price[:len(Real_Price)-prediction_days]
df_test= Real_Price[len(Real_Price)-prediction_days:]


In [8]:
# Data preprocess
training_set = df_train.values
training_set = np.reshape(training_set, (len(training_set), 1))
from sklearn.preprocessing import MinMaxScaler
sc = MinMaxScaler()
training_set = sc.fit_transform(training_set)
X_train = training_set[0:len(training_set)-1]
y_train = training_set[1:len(training_set)]
X_train = np.reshape(X_train, (len(X_train), 1, 1))


In [9]:
# Importing the Keras libraries and packages
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

# Initialising the RNN
regressor = Sequential()

# Adding the input layer and the LSTM layer
regressor.add(LSTM(units = 4, activation = 'sigmoid', input_shape = (None, 1)))

# Adding the output layer
regressor.add(Dense(units = 1))

# Compiling the RNN
regressor.compile(optimizer = 'adam', loss = 'mean_squared_error')

# Fitting the RNN to the Training set
regressor.fit(X_train, y_train, batch_size = 5, epochs = 100)


Epoch 1/100
2167/2167 [==============================] - 3s 2ms/step - loss: 0.0526
Epoch 2/100
2167/2167 [==============================] - 1s 629us/step - loss: 0.0079
Epoch 3/100
2167/2167 [==============================] - 1s 627us/step - loss: 0.0072
Epoch 4/100
2167/2167 [==============================] - 1s 614us/step - loss: 0.0065
Epoch 5/100
2167/2167 [==============================] - 1s 663us/step - loss: 0.0058
Epoch 6/100
2167/2167 [==============================] - 1s 623us/step - loss: 0.0051
Epoch 7/100
2167/2167 [==============================] - 1s 616us/step - loss: 0.0043
Epoch 8/100
2167/2167 [==============================] - 1s 687us/step - loss: 0.0035
Epoch 9/100
2167/2167 [==============================] - 1s 648us/step - loss: 0.0027
Epoch 10/100
2167/2167 [==============================] - 2s 706us/step - loss: 0.0019
Epoch 11/100
2167/2167 [==============================] - 1s 684us/step - loss: 0.0012
Epoch 12/100
2167/2167 [==============================

In [10]:
# Making the predictions
test_set = df_test.values
inputs = np.reshape(test_set, (len(test_set), 1))
inputs = sc.transform(inputs)
inputs = np.reshape(inputs, (len(inputs), 1, 1))
predicted_BTC_price = regressor.predict(inputs)
predicted_BTC_price = sc.inverse_transform(predicted_BTC_price)


In [ ]:
# Visualising the results
plt.figure(figsize=(25,15), dpi=80, facecolor='w', edgecolor='k')
ax = plt.gca()  
plt.plot(test_set, color = 'red', label = 'Real BTC Price')
plt.plot(predicted_BTC_price, color = 'blue', label = 'Predicted BTC Price')
plt.title('BTC Price Prediction', fontsize=40)
df_test = df_test.reset_index()
x=df_test.index
labels = df_test['date']
plt.xticks(x, labels, rotation = 'vertical')
for tick in ax.xaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
for tick in ax.yaxis.get_major_ticks():
    tick.label1.set_fontsize(18)
plt.xlabel('Time', fontsize=40)
plt.ylabel('BTC Price(USD)', fontsize=40)
plt.legend(loc=2, prop={'size': 25})
plt.show()
